# FunOS Malloc report

This notebook summarizes mcache metrics.

In [ ]:
# *NOTE*: run this command to clean output cell and meta data.

#nb-clean clean  ./funos_stats_analysis/malloc_report.ipynb   

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.ticker as mticker
import numpy as np
plt.rcParams['figure.figsize'] = [15, 25]

#https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
from IPython.display import display, Markdown, Latex

In [ ]:
df_filename_pattern = "malloc_caches_slot_stats_slot_{}.pkl"

In [ ]:
slots = [s for s in range(6,16)]
slot_idx = [i for i in range(len(slots))]

In [ ]:
df_filenames = [df_filename_pattern.format(s) for s in slots]

In [ ]:
dfs = [pd.read_pickle(df_filename) for df_filename  in df_filenames]

In [ ]:
def plot_vp_index(df, col, title):
    plt.title(title)
    plt.scatter(df.index, df[col])
    plt.xticks(rotation = 70)
    plt.grid()
    plt.show()

In [ ]:
def plot_vp_index_all_slots(dfs, col, title, height=30):
    note_str = '## y axis: **{}**'.format(title)
    display(Markdown(note_str))
    note_str = "## x axis: **cluster,core number (`cluster.core`)**"
    display(Markdown(note_str))

    # use short notation for cluster.core for x-axis
    short_vp_names = ["{}.{}".format(vp.split('_')[1], vp.split('_')[2]) for vp in dfs[0].index]

    # prepare subplot
    fig, axs = plt.subplots(len(slots),1)
    fig.set_figheight(height)

    for i, s in zip(slot_idx, slots):
        df = dfs[i]
        if col == "Avail":
            max_val = df['Max'][0]
            # axs[i].plot(df.index, df['Max'], 'r', label='Max')
            axs[i].axhline(y=max_val, color='r', linestyle='-') #, label="Max(limit)")
            axs[i].scatter(df.index, df[col], label=col)
            axs[i].plot(df.index, df['Avail(avg)'], color='y', label="Avail(avg)")
            axs[i].errorbar(df.index, df['Avail(avg)'], yerr=df['Avail(std)'], color='y', fmt=".") #, label="Avail(std)")
            # axs[i].axhline(y=df[col].mean(), color='y', linestyle='-', label="Avail(Mean)")
            # axs[i].errorbar(df.index, df[col], yerr=df[col].std(), fmt="-", label="Avail(std)")

            axs[i].plot(df.index, df['Avail(max)'], 'g', label='Avail(max)')
            # 'Avail(min) is reset to max, so if == max, then not yet updated so set to itself
            avail_min = np.where(df['Avail(min)'] != df['Max'], df['Avail(min)'], df[col])
            # avail_min = df['Avail(min)']
            axs[i].plot(df.index, avail_min, 'b', label='Avail(min)')
            axs[i].plot(df.index, df['Repl_th_val'], '.', label='Repl_th_val')
            axs[i].set_ylim([0, max_val*1.05])
            if i == 0: print("Red line: max")
        elif col in ['Hit']:
            if i == 0:
                note_str = "*NOTE*: {}, y axis is $\log$ scale.".format(col)
                display(Markdown(note_str))
            axs[i].scatter(df.index, df[col], label="Hit (log)")
            axs[i].set_yscale('log')
        else:
            axs[i].scatter(df.index, df[col], label=col)
        # axs[i].scatter(df.index, df_1[col])
        axs[i].set_ylabel("Slot: {}".format(s))
        axs[i].grid()
        axs[i].legend()
        ticks_loc = axs[i].get_xticks()
        axs[i].xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
        axs[i].set_xticklabels(short_vp_names, rotation=70)
        axs[i].set_xlabel('cluster.core')
    
    plt.show()

## Malloc stats for all slots

In [ ]:
print("Column list: {}".format(dfs[0].columns))

In [ ]:
black_list = ['Avail Bytes', 'Avail(max)', 'Avail(min)', 'Max repl. time (nsec)', 'MIN repl. miss inter dur. (nsec)',
       'MIN alloc inter dur. (nsec)', 'Max']
print("Will plot except these columns: {}".format(black_list))

In [ ]:
for col in dfs[0].columns:
    if col not in black_list: 
        plot_vp_index_all_slots(dfs, col, col)